In [1]:
#IMPORTATIONS#

# core importations
import caveclient
from caveclient import CAVEclient
import cloudvolume
from cloudvolume import CloudVolume
import pandas as pd
import numpy as np

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

In [2]:
#FUNCTIONS#

def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    x[0] = str(x[0])
    x[1] = str(x[1])
    x[2] = str(x[2])
    return x

# defines function to convert list of coordinates in [4,4,40] resolution to root id #
def coordsToRoot(coords):

    # sets client #
    client = CAVEclient("flywire_fafb_production")

    # sets cloud volume #
    cv = cloudvolume.CloudVolume("graphene://https://prod.flywire-daf.com/segmentation/1.0/fly_v31", use_https=True)

    # determines resolution of volume #
    res = cv.resolution

    # converts coordinates using volume resolution #
    cv_xyz = [int(coords[0]/(res[0]/4)),int(coords[1]/(res[1]/4)),int(coords[2]/(res[2]/40))]

    # sets point by passing converted coordinates into 'download_point' method #
    point = int(cv.download_point(cv_xyz, size=1))

    # looks up root id for that supervoxel using chunkedgraph #
    root_result = client.chunkedgraph.get_root_id(supervoxel_id=point)

    return root_result

def getNucs(ids, query_type):
    
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #gets current materialization version#
    mat_vers = max(client.materialize.get_versions())
    
    #pulls nucleus table results based on query type#
    if query_type == 'coord':
        nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": coordsToRoot(ids)},
                                            materialization_version = mat_vers)
    elif query_type == 'nuc':
        nuc_df = client.materialize.query_table('nuclei_v1',
                                                filter_in_dict={"id": nucleus_ids},
                                                materialization_version = mat_vers)
    elif query_type == 'root':
        nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": ids},
                                            materialization_version = mat_vers)
        
    nuc_df['pt_position'] = [coordConvert(i) for i in nuc_df['pt_position']]
    
    
    
    #creates output dataframe using root id, nuc id, and nuc coords from table to keep alignment#
    out_df = pd.DataFrame({'Root ID':list(nuc_df['pt_root_id'].astype('str')), #converts ids to str to avoid rounding#
                           'Nucleus ID':list(nuc_df['id']),
                           'Nucleus Coordinates':list(nuc_df['pt_position'])})
        
    return out_df

# defines function to get presynaptic table entry using root id
def getSyns(root_ids):
    
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #gets current materialization version#
    mat_vers = max(client.materialize.get_versions())
    
    #creates blank output dataframe#
    out_df = pd.DataFrame(columns = ['Root ID','Incoming Synapses','Outgoing Synapses',
                                     'Upstream Parters','Downstream Partners'])
    
    
    #iterates through root ids, creating df rows and adding them to output df#
    for i in root_ids:
    
        #gets pre and post synapse tables using root id#
        pre_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                                filter_in_dict={"pre_pt_root_id":[i]})
        post_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                                filter_in_dict={"post_pt_root_id":[i]})

        #counts total pre and post synapses#
        pre_num = len(pre_syn_df)
        post_num = len(post_syn_df)

        #gets lists of pre and post synaptic partners#
        downstream_num = len(pre_syn_df['post_pt_root_id'].unique())
        upstream_num = len(post_syn_df['pre_pt_root_id'].unique())

        #makes row dataframe#
        row_df = pd.DataFrame({'Root ID': [i],'Incoming Synapses':[post_num],'Outgoing Synapses':[pre_num],
                               'Upstream Parters':[upstream_num],'Downstream Partners':[downstream_num]})
        
        #appends row onto output df
        out_df = out_df.append(row_df, ignore_index = True)
    
    return out_df

# defines function to get edit data
def getEdits(root_ids):
    
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #creates blank output dataframe#
    out_df = pd.DataFrame()
    
    #iterates through root ids to fill output df#
    for i in root_ids:
        
        #attempts to get changelog and assign edit values using root id#
        try:
            #gets changelog dictionary using root id#
            change_dict = client.chunkedgraph.get_change_log(i)

            #sets edit data objects#
            splits = change_dict['n_splits']
            merges = change_dict['n_mergers']
            edits = splits + merges
        
        #handles exception if query returns error#
        except:
            splits = 'n/a'
            merges = 'n/a'
            edits = 'n/a'
        
        #creates row df from edit data objects#
        row_df = pd.DataFrame({'Root ID': [i],'Splits':[splits],'Merges':[merges],
                               'Total Edits':[edits]})
        
        #appends row onto output df, MAY NEED TO MAKE IGNORE INDEX = TRUE#
        out_df = out_df.append(row_df)

    return out_df   

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilder(id_list, options):
    
    #automatically determines data type based on length of input ids#
    if all([len(str(i)) == 18 for i in id_list]):
        query_type = 'root'
    elif all([len(str(i)) == 7 for i in id_list]):
        query_type = 'nuc'
    elif len(id_list) % 3 == 0:
        query_type = 'coord'
    
    #creates output dataframe using nuc df as base to align ids#
    out_df = getNucs(id_list,query_type)
    
    #creates root list based on (potentially) converted values from out_df#
    root_list = list(out_df['Root ID'])
    
    #adds synapse data if synapse checkbox is marked#
    if 'syns' in options:
        
        #creates syn_df using root list#
        syn_df = getSyns(root_list)
        
        #joins to out_df#
        out_df = out_df.join(syn_df.set_index('Root ID'), on='Root ID')
    
    #adds edit data if edits checkbox is marked#
    if 'edits' in options: #CHANGE THIS TO TABULAR LOG#
        
        #creates edit_df using root list#
        edit_df = getEdits(root_list)
        
        #joins to out_df#
        out_df = out_df.join(edit_df.set_index('Root ID'), on='Root ID')
          
    return out_df 

In [3]:
# DASH APP #

app = dash.Dash(__name__)

# defines layout of various app elements (submission field, checkboxes, button, output table)#
app.layout = html.Div([
    dcc.Textarea(
        id='message_text',
        value='Input coordinates, select output parameters, and click "Submit" button.\n'\
        'ID queries are limited to 20 entries, coordinate lookups must be done one at a time.\n'\
        'Lookup takes ~2-3 seconds per entry.',
        style={'width': '800px','resize': 'none'},
        rows=3,
        disabled=True,
    ),
    html.Div(dcc.Input(  #defines input field#
        id='input_field', 
        type='text', 
        placeholder='ID Number',
    )),
    html.Br(
    ),
    html.Div(dcc.Checklist(  #defines data selection checkboxes#
        id='checkboxes',
        options=[
            {'label': 'Synapse Count', 'value': 'syns'},
            {'label': 'Edits', 'value': 'edits'},
        ],
        labelStyle={'display': 'block'},
        value=['syns','edits'],
    )),
    html.Br(
    ),
    html.Button(  #defines submission button#
        'Submit', 
        id='submit_button', 
        n_clicks=0,
    ),
    html.Br(
    ),
    html.Div(dash_table.DataTable(  #defines output table#
        id='table', 
        fill_width=False,  #sets column width to fit text instead of expanding to container width#
        export_format="csv",
    ))
])

# defines callback that takes root ids and desired data selection on button click and generates table
@app.callback(
    Output('table','columns'),           #defines first output location as the 'columns' aspect of 'table'#
    Output('table', 'data'),             #defines second output location as the 'data' aspect of 'table'#
    Output('message_text','value'),      #defines second output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'),  #defines trigger as button press# 
    State('input_field', 'value'),       #defines first input state as the value of 'input_field'#
    State('checkboxes','value'),         #defines second input state as the value of 'checkboxes'#
    prevent_initial_call=True,           #prevents function from executing prior to button press#
)
def update_output(n_clicks, ids, checked):
    
    #splits 'roots' string into list#
    id_list = str(ids).split(",")   
    
    #strips spaces from root_list entries and converts to integers#
    id_list = [int(x.strip(' ')) for x in id_list] 
    
    #if query list under 20 items, generates dataframe#
    if len(id_list) <= 20:
        
        #creates df using dfBuilder function#
        df = dfBuilder(id_list, checked)
        
        df['Nucleus Coordinates'] = df['Nucleus Coordinates'].astype('str')
        
        #creates column list based on dataframe columns#
        column_list = [{"name": i, "id": i} for i in df.columns]
        
        #converts df to dictionary#
        df_dict =  df.to_dict('records')
        
        #keeps message output the same#
        message_output = 'Choose lookup method from dropdown, input coordinates, select output parameters, '\
        'and click "Submit" button.\nID queries are limited to 20 entries, coordinate lookups must be done one at a time.'
        
        #builds output list#
        output_list = [column_list,df_dict,message_output]        #combines df_dict and column_list into output list#
        
        return output_list                                        #returns list of column names and data values#
    
    #returns error message if query list is longr than 20 items#
    else:
        return [0,0,'Please limit each query to a maximum of 20 items.']
        

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2021 14:08:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 14:08:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 14:08:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 14:08:59] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 14:08:59] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 14:09:07] "POST /_dash-update-component HTTP/1.1" 200 -
